# Hierarchical model


In [24]:
import sympy

from mira.metamodel import *
from mira.modeling import Model
from mira.modeling.amr.petrinet import template_model_to_petrinet_json
from mira.sources.amr import model_from_json
from pyro.infer.inspect import get_dependencies
from pyciemss.compiled_dynamics import CompiledDynamics
import torch
import json
from chirho.dynamical.handlers.solver import TorchDiffEq
from pyciemss.compiled_dynamics import (
    _compile_param_values,
    get_name,
)
def test_distribution_expressions():
    beta_mean = Parameter(name='beta_mean',
                          distribution=Distribution(type="Beta1",
                                                    parameters={'alpha': sympy.Integer(1),
                                                                'beta': sympy.Integer(10)}))
    gamma_mean = Parameter(name='gamma_mean',
                           distribution=Distribution(type="Beta1",
                                                     parameters={'alpha': sympy.Integer(10),
                                                                 'beta': sympy.Integer(10)}))
    beta = Parameter(name='beta',
                     distribution=Distribution(type="InverseGamma1",
                                               parameters={'shape': sympy.Symbol('beta_mean'),
                                                           'scale': sympy.Float(0.01)}))
    gamma = Parameter(name='gamma',
                      distribution=Distribution(type="InverseGamma1",
                                                parameters={'shape': sympy.Symbol('gamma_mean'),
                                                            'scale': sympy.Float(0.01)}))

    # Make an SIR model with beta and gamma in rate laws
    sir_model = TemplateModel(
        templates=[
            ControlledConversion(
                subject=Concept(name='S'),
                outcome=Concept(name='I'),
                controller=Concept(name='I'),
                rate_law=sympy.Symbol('S') * sympy.Symbol('I') * sympy.Symbol('beta')
            ),
            NaturalConversion(
                subject=Concept(name='I'),
                outcome=Concept(name='R'),
                rate_law=sympy.Symbol('I') * sympy.Symbol('gamma')
            ),
        ],
        parameters={
            'beta': beta,
            'gamma': gamma,
            'beta_mean': beta_mean,
            'gamma_mean': gamma_mean,
            }
    )

    model = Model(sir_model)
    pn_json = template_model_to_petrinet_json(sir_model)
    with open('hierarchical_sir_model.json', 'w') as fh:
        json.dump(pn_json, fh)
        
    params = pn_json['semantics']['ode']['parameters']
    assert {p['id'] for p in params} == \
        {'beta_mean', 'gamma_mean', 'beta', 'gamma'}
    beta = [p for p in params if p['id'] == 'beta'][0]
    assert beta['distribution']['type'] == 'InverseGamma1'
    assert beta['distribution']['parameters']['shape'] == 'beta_mean'

    # Now read the model back and check if it is deserialized
    tm = model_from_json(pn_json)
    assert tm.parameters['beta'].distribution.type == 'InverseGamma1'
    assert isinstance(tm.parameters['beta'].distribution.parameters['shape'],
                      SympyExprStr)
    assert tm.parameters['beta'].distribution.parameters['shape'].args[0] == \
        sympy.Symbol('beta_mean')
    return sir_model


# Compile the non-hierarchical model and get dependencies

In [21]:
url_sirhd = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/refs/heads/main/data/models/sirhd.json"
compiled_model = CompiledDynamics.load(url_sirhd)
with TorchDiffEq():
    #simulation = compiled_model(start_time=torch.as_tensor(0.0), end_time=torch.as_tensor(0.0))
    dependencies = get_dependencies(compiled_model, model_args=[torch.as_tensor(0.0), torch.as_tensor(0.0)])
dependencies

{'prior_dependencies': {'persistent_beta': {'persistent_beta': set()},
  'persistent_gamma': {'persistent_gamma': set()}},
 'posterior_dependencies': {'persistent_beta': {'persistent_beta': set()},
  'persistent_gamma': {'persistent_gamma': set()}}}

# Compile the hierarchical model and get dependencies

In [ ]:
sir

## Get all the mira parameters and their dependencies before we compile the dynamics

In [23]:
sir_model = test_distribution_expressions()
sir_model.parameters

{'beta': Parameter(name='beta', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='InverseGamma1', parameters={'shape': beta_mean, 'scale': 0.01})),
 'gamma': Parameter(name='gamma', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='InverseGamma1', parameters={'shape': gamma_mean, 'scale': 0.01})),
 'beta_mean': Parameter(name='beta_mean', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='Beta1', parameters={'alpha': 1, 'beta': 10})),
 'gamma_mean': Parameter(name='gamma_mean', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='Beta1', parameters={'alpha': 10, 'beta': 10}))}

## Get the dependencies in the mira model

In [ ]:
def get_dependencies(src: TemplateModel) -> dict:
    for param_info in src.parameters.values():
        param_name = get_name(param_info)

        if param_info.placeholder:
            continue

        param_dist = getattr(param_info, "distribution", None)
        if param_dist is None:
            param_value = param_info.value
        for :
            param_value = recursive_mira_distribution_to_pyro(param_dist)


### Create a recursive mira distributions to pyro
However, I think we want to go one step above this so that we can determine the dependencies before we call MIRA_TO_PYRO.

In [25]:
from pyciemss.mira_integration.distributions import _MIRA_TO_PYRO, _TESTED_DISTRIBUTIONS
import mira
def recursive_mira_distribution_to_pyro(mira_dist: mira.metamodel.Distribution):

    if mira_dist.type not in _MIRA_TO_PYRO.keys():
        raise NotImplementedError(
            f"Conversion from MIRA distribution type {mira_dist.type} to Pyro distribution is not implemented."
        )

    if mira_dist.type not in _TESTED_DISTRIBUTIONS:
        warnings.warn(
            f"Conversion from MIRA distribution type {mira_dist.type} to Pyro distribution has not been tested."
        )

    simple_parameters = {k: torch.as_tensor(v)
                   for k, v in mira_dist.parameters.items()
                   if not isinstance(v, mira.metamodel.SymPyExprStr)}
    

    return _MIRA_TO_PYRO[mira_dist.type](simple_parameters)



In [18]:
url_hierarchical = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/refs/heads/main/data/models/hierarchical_sir_model.json"
compiled_model = CompiledDynamics.load(url_hierarchical)
with TorchDiffEq():
    #simulation = compiled_model(start_time=torch.as_tensor(0.0), end_time=torch.as_tensor(0.0))
    dependencies = get_dependencies(compiled_model, model_args=[torch.as_tensor(0.0), torch.as_tensor(0.0)])
dependencies

/Users/zuck016/Projects/ASKEM/build/pyciemss/pyciemss/mira_integration/distributions.py:257: UserWarning: Conversion from MIRA distribution type InverseGamma1 to Pyro distribution has not been tested.
  warnings.warn(


ValueError: The model parameters could not be compiled. Please check the model definition.